In [1]:
import numpy as np

In [2]:
def GF_table(x, bc_left: str, bc_right: str, L, truncation):
    # returns the eigenvalue beta_m and the norm Nx

    m = np.arange(1, truncation+1)
    if (bc_left == 'D' and bc_right == 'D'): #11
        beta_m = m*np.pi/L
        # X_m = np.sin(beta_m*x)
        Nx = L/2
        return beta_m, Nx
    
    if (bc_left == 'D' and bc_right == 'N'): #12
        beta_m = (2*m-1)*np.pi/(2*L)
        # X_m = np.sin(beta_m*x)
        Nx = L/2
        return beta_m, Nx
    

def green_function(x, y, x_s, y_s, L, bc_e: str, bc_n: str, bc_w: str, bc_s: str, truncation = 10):
    beta_m, Nx = GF_table(x, bc_e, bc_w, L, truncation)
    theta_n, Ny = GF_table(y, bc_s, bc_n, L, truncation)
    summation_array = np.zeros(truncation*truncation)
    
    for m in range(1, truncation+1):
        i = m-1
        for n in range(1, truncation+1):
            j= n-1

            X_m = np.sin(x*beta_m[i])
            X_m_s = np.sin(x_s*beta_m[i])
            print("X_m_s: " + str(X_m_s))
            Y_n = np.sin(y*theta_n[j])
            Y_n_s = np.sin(y_s*theta_n[j])

            fraction = 1/((beta_m[i]*beta_m[i]) + (theta_n[j]*theta_n[j]))
            X_mult = (X_m*X_m_s)/Nx
            Y_mult = (Y_n*Y_n_s)/Ny

            summation_array[i*truncation + j] = fraction * np.dot(X_mult, Y_mult)
    
    sum = np.sum(summation_array)
    return sum

In [3]:
L = 1
x_s = y_s = L/2
x_in = np.arange(0, L/2, 0.1)
x_out = np.arange(L/2, L, 0.1)
y_in = np.arange(0, L/2, 0.1)
y_out = np.arange(L/2, L, 0.1)

green_function(x_in, y_in, x_s, y_s, L, bc_e = "D", bc_n = "D", bc_w = "D", bc_s = "D")
# green_function(x, y, x_s, y_s, L, bc_e: str, bc_n: str, bc_w: str, bc_s: str, truncation = 10):

X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.2246467991473532e-16
X_m_s: 1.2246467991473532e-16
X_m_s: 1.2246467991473532e-16
X_m_s: 1.2246467991473532e-16
X_m_s: 1.2246467991473532e-16
X_m_s: 1.2246467991473532e-16
X_m_s: 1.2246467991473532e-16
X_m_s: 1.2246467991473532e-16
X_m_s: 1.2246467991473532e-16
X_m_s: 1.2246467991473532e-16
X_m_s: -1.0
X_m_s: -1.0
X_m_s: -1.0
X_m_s: -1.0
X_m_s: -1.0
X_m_s: -1.0
X_m_s: -1.0
X_m_s: -1.0
X_m_s: -1.0
X_m_s: -1.0
X_m_s: -2.4492935982947064e-16
X_m_s: -2.4492935982947064e-16
X_m_s: -2.4492935982947064e-16
X_m_s: -2.4492935982947064e-16
X_m_s: -2.4492935982947064e-16
X_m_s: -2.4492935982947064e-16
X_m_s: -2.4492935982947064e-16
X_m_s: -2.4492935982947064e-16
X_m_s: -2.4492935982947064e-16
X_m_s: -2.4492935982947064e-16
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 1.0
X_m_s: 3.6739403974420594e-16
X_m_s: 3.67394039744

0.37480904676447585

In [ ]:
import numpy as np

# Define the function f(i, j)
def f(i, j):
    return 1 / (i * j)

# Define the range for i and j
N = 10  # Adjust N as needed
M = 10  # Adjust M as needed

# Create arrays for i and j
i_values = np.arange(1, N + 1)
j_values = np.arange(1, M + 1)

# Create a grid of i and j values
I, J = np.meshgrid(i_values, j_values, indexing='ij')

# Evaluate the function on the grid
F = f(I, J)

# Perform the double summation
S = np.sum(F)

print(f"The result of the double summation is {S}")


In [ ]:
beta_m = 2
theta_n = 3
Nx=2
Ny=3
x = np.arange(0, L, 0.1)
y = np.arange(0, L, 0.1)

In [ ]:
X_m = np.sin(x*beta_m)
X_m_s = np.sin(x_s*beta_m)
Y_n = np.sin(y*theta_n)
Y_n_s = np.sin(y_s*theta_n)

In [ ]:
fraction = 1/((beta_m*beta_m) + (theta_n*theta_n))
X_mult = (X_m*X_m_s)/Nx
Y_mult = (Y_n*Y_n_s)/Ny

In [ ]:
np.dot(X_mult, Y_mult)

In [ ]:
import numpy as np

In [ ]:
import numpy as np

# Define the domain
x = np.linspace(0, 2, 1000)  # 1000 points between 0 and 2

# Define the function to be integrated
y = x  # since we are integrating x

# Perform the integration using the trapezoidal rule
integral = np.trapz(y, x)

print(f"The integral of x from 0 to 2 is approximately {integral}")


In [ ]:
import numpy as np

# Define the function to be integrated
def f(x, y):
    return x * y

# Define the domain for x and y
x = np.linspace(0, 2, 100)
y = np.linspace(0, 3, 100)

# Create a grid of points
X, Y = np.meshgrid(x, y)

# Evaluate the function on the grid
Z = f(X, Y)

# Perform the double integration using the trapezoidal rule
# First, integrate with respect to y for each fixed x
integral_y = np.trapz(Z, y, axis=0)

# Then, integrate the result with respect to x
integral = np.trapz(integral_y, x)

print(f"The double integral of xy over [0, 2] x [0, 3] is approximately {integral}")
